In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#https://huggingface.co/ckiplab/gpt2-base-chinese
model_name = "ckiplab/gpt2-base-chinese"

# 檢查是否有 GPU 可以用（如果有會快很多）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用裝置：{device}")

使用裝置：cuda


In [3]:
# 載入指定的語言模型與 tokenizer
# model_name 是你指定的模型名稱，例如 "ckiplab/gpt2-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 用於將文字轉換為模型輸入的 token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # 載入因果語言模型並指定裝置（例如 CPU 或 GPU）

# 定義一個函式，用來根據提示詞（prompt）與選擇的風格生成文字
def generate_text(prompt, style="default", max_new_tokens=150, temperature=0.9, top_k=50, top_p=0.95):
    # 定義可選的風格標籤字典，對應不同的語氣或文體
    style_labels = {
        "default": "一般",
        "fairy_tale": "童話",
        "ancient_chinese": "古文",
        "child_style": "兒童",
        "mystery": "懸疑",
        "sci_fi": "科幻",
        "wuxia": "武俠",
        "diary": "日記",
        "news": "新聞",
        "poetic": "詩詞"
    }

    # 根據 style 參數取得對應的中文風格標籤，若未指定則預設為「一般」
    style_tag = style_labels.get(style, "一般")

    # 將風格標籤加入提示詞中，以引導模型產生指定風格的內容
    styled_prompt = f"[風格: {style_tag}]\n{prompt}"

    # 將 styled_prompt 編碼成模型可接受的 token ID，並轉成 tensor
    input_ids = tokenizer.encode(styled_prompt, return_tensors="pt").to(device)

    # 建立 attention mask，避免模型注意到 padding 的位置（這裡預設沒有 padding，但為保險起見仍處理）
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # 使用模型產生文字
    output_ids = model.generate(
        input_ids,                      # 輸入的 token
        attention_mask=attention_mask, # 注意力遮罩，讓模型知道哪些位置是有效輸入
        max_new_tokens=max_new_tokens, # 生成的最大 token 數量
        temperature=temperature,       # 溫度值控制隨機性，越高越多樣化
        top_k=top_k,                   # top-k 篩選（僅在前 k 個最可能的詞中選擇）
        top_p=top_p,                   # top-p (nucleus sampling)，保留機率總和為 p 的詞集合中選擇
        do_sample=True,                # 啟用隨機取樣（非貪婪解碼）
        pad_token_id=tokenizer.eos_token_id  # 設定 pad token 為結束符號，以避免警告
    )

    # 解碼模型輸出的 token ID，轉回可讀文字，並移除多餘空白
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).replace(" ", "")
    return output_text  # 回傳最終生成的文字

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

In [4]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "fairy_tale"
# 呼叫前面定義的 generate_text 函式，傳入提示文字與風格，產生對應風格的續寫文字
result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： fairy_tale
產生的故事： [風格:童話]故事開始於一個神秘的森林，小狐狸出現在月光下。了從不同神祕主人身上逃走，原本是成為一個生活在極短的世界，但由於他們過於激烈，故後來漸漸成為一個被稱為「邪惡人」（）的人。有一天，這個邪惡人在森林裡出現。他們一天發現了他們的手指，他們的手指被綁在了一個洞裡，這種小狐狸的身體很像它的身體。在過去，它從未出現過，只是因為那時天氣變壞，在這時開始有了很多


In [5]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "sci_fi"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： sci_fi
產生的故事： [風格:科幻]故事開始於一個神秘的森林，小狐狸出現在月光下。們的大腿部的大腿很少，但牠們在這裡的聲音很大。他們的前臂部分會由兩個手臂組成，一個是大拇指（腳）和一個是大拇指（腳）。然後在其他地方的前臂，會經由下頜，在一種叫做（小巫女），這個手指比大拇指多了三個腳。在這些地方，他們還會在地中海岸發現牠們。這可能是牠們從阿波羅（尤其是在地中海）移動至地中海一


In [6]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "news"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： news
產生的故事： [風格:新聞]故事開始於一個神秘的森林，小狐狸出現在月光下。世紀的《神祕的世界》（）在漫畫版本中，描繪了一個自己的男子。《神祕的世界》（）是《神祕的世界》的第二集，第三集則是第三集。在漫畫版本中，主角的死亡也不是真正的世界，不是任何人。故事的起源可以追溯到《神祕的世界》（of）。故事裡的魔法師是來自神祕的一支神祕的，但是主角的死亡是在魔法師的幫助下才


In [7]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "wuxia"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： wuxia
產生的故事： [風格:武俠]故事開始於一個神秘的森林，小狐狸出現在月光下。說家裡面有一個主人，他是老婆，但老婆說他不喜歡這個人，而且他的主人要做女人，不喜歡這個人。老婆說這個故事是天地所不能的，當時他要開始想要一個女人，這個故事並不是他個人的故事。但是有一個主人說他不喜歡這個故事，因為他是神祕的人。一些故事家裡有一個神祕的人，他是老婆。這個故事也叫做神祕的人，他是神祕的人
